In [40]:
import pandas as pd
from nltk import word_tokenize
import re
import nltk
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.svm import LinearSVC
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

**Preprocessing Data**

In [2]:
df=pd.read_csv("p2_train.csv")
df_test=pd.read_csv("p2_test.csv")
df_question=df["question"]
df_response=df["response"]
df_question_test=df_test["question"]
df_response_test=df_test["response"]


In [3]:
def preprocessor(text):
  review=re.sub('\[[^]]*\]', ' ', str(text))
  review=re.sub('[^a-zA-z]', ' ', str(text))
  review=review.lower().split()
  return review
df_question=df_question.apply(preprocessor)
df_response=df_response.apply(preprocessor)
df_question_test=df_question_test.apply(preprocessor)
df_response_test=df_response_test.apply(preprocessor)

**POS Tagging**

In [4]:
lst=[]
for index, row in df_question.to_frame().iterrows():
  for i in row:
    string=" "
    string=string.join(i)
    tokens=word_tokenize(string)
    lst.append(nltk.pos_tag(tokens))
lst1=[]
for index, row in df_response.to_frame().iterrows():
  for i in row:
    string=" "
    string=string.join(i)
    tokens=word_tokenize(string)
    lst1.append(nltk.pos_tag(tokens))



In [5]:
lst_test=[]
for index, row in df_question_test.to_frame().iterrows():
  for i in row:
    i=i[1:len(i)-1]
    string=" "
    string=string.join(i)
    tokens=word_tokenize(string)
    lst_test.append(nltk.pos_tag(tokens))
lst1_test=[]
for index, row in df_response_test.to_frame().iterrows():
  for i in row:
    i=i[1:len(i)-1]
    string=" "
    string=string.join(i)
    tokens=word_tokenize(string)
    lst1_test.append(nltk.pos_tag(tokens))

**Combining words with tags**

In [6]:
res=[]
for i in lst:
  combined_list=[]
  for j in i:
    combined_list.append(j[0]+"/"+j[1])
  res.append(combined_list)
res1=[]
for i in lst1:
  combined_list=[]
  for j in i:
    combined_list.append(j[0]+"/"+j[1])
  res1.append(combined_list)
res_test=[]
for i in lst_test:
  combined_list=[]
  for j in i:
    combined_list.append(j[0]+"/"+j[1])
  res_test.append(combined_list)
res1_test=[]
for i in lst1_test:
  combined_list=[]
  for j in i:
    combined_list.append(j[0]+"/"+j[1])
  res1_test.append(combined_list)
   

In [7]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=f1d760623604a52884b0c3e121d2f301cbb85a1c6d59d42d854ffc034d6b79f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-pv40by8n/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [8]:
import spacy
import en_core_web_lg
nlp=en_core_web_lg.load()

**Finding vectors using Spacy**

In [9]:
def get_vector(x):
    doc=nlp(x)
    vec=doc.vector
    return vec

temp1,temp2=[],[]
for i in res:
    string=" "
    string=string.join(i)
    temp1.append(string)
for i in res1:
    string=" "
    string=string.join(i)
    temp2.append(string)
question,response=pd.Series(temp1),pd.Series(temp2)
question=question.apply(lambda x: get_vector(x))
response=response.apply(lambda x: get_vector(x))
question,response=question.to_numpy(),response.to_numpy()
question,response=question.reshape(-1,1),response.reshape(-1,1)
question=np.concatenate(np.concatenate(question,axis=0),axis=0).reshape(-1,300)
response=np.concatenate(np.concatenate(response,axis=0),axis=0).reshape(-1,300)


In [10]:
final_vector=np.concatenate((question,response),axis=1)
final_vector.shape

(1640, 600)

In [11]:
temp1,temp2=[],[]
for i in res_test:
    string=" "
    string=string.join(i)
    temp1.append(string)
for i in res1_test:
    string=" "
    string=string.join(i)
    temp2.append(string)
question,response=pd.Series(temp1),pd.Series(temp2)
question=question.apply(lambda x: get_vector(x))
response=response.apply(lambda x: get_vector(x))
question,response=question.to_numpy(),response.to_numpy()
question,response=question.reshape(-1,1),response.reshape(-1,1)
question=np.concatenate(np.concatenate(question,axis=0),axis=0).reshape(-1,300)
response=np.concatenate(np.concatenate(response,axis=0),axis=0).reshape(-1,300)

In [12]:
final_vector_test=np.concatenate((question,response),axis=1)
final_vector_test.shape


(410, 600)

**Classifiers**

In [13]:
X_train,X_test,y_train,y_test=final_vector,final_vector_test,df["type"],df_test["type"]

In [14]:
clf = RandomForestClassifier(n_estimators=15, random_state=10,class_weight="balanced")
clf.fit(X_train,y_train)
predictions=clf.predict(X_test)
print("Accuracy is",accuracy_score(predictions,y_test))
print("F1 Score is",f1_score(predictions,y_test,average='weighted'))
print("Precision Score is",precision_score(predictions,y_test,average="weighted"))
print("Recall Score is",recall_score(predictions,y_test,average="weighted"))


Accuracy is 0.7609756097560976
F1 Score is 0.8219955654101998
Precision Score is 0.9012382739212009
Recall Score is 0.7609756097560976


In [15]:
print(confusion_matrix(predictions,y_test, labels=["answered", "attacked", "irrelevant","agreed"]))

[[306  35  32  11]
 [  2   2   2   0]
 [ 12   1   4   2]
 [  0   1   0   0]]


**Sentence Embeddings(Spacy)**

In [16]:
que=[]
for i in df_question:
  que.append(i)
resp=[]
for i in df_response:
  resp.append(i)
que_test=[]
for i in df_question_test:
  que_test.append(i)
resp_test=[]
for i in df_response_test:
  resp_test.append(i)
  

In [17]:
def get_vector(x):
    doc=nlp(x)
    vec=doc.vector
    return vec
temp1,temp2=[],[]
for i in que:
    string=" "
    string=string.join(i)
    temp1.append(string)
for i in resp:
    string=" "
    string=string.join(i)
    temp2.append(string)
question,response=pd.Series(temp1),pd.Series(temp2)
question=question.apply(lambda x: get_vector(x))
response=response.apply(lambda x: get_vector(x))
question,response=question.to_numpy(),response.to_numpy()
question,response=question.reshape(-1,1),response.reshape(-1,1)
question_train=np.concatenate(np.concatenate(question,axis=0),axis=0).reshape(-1,300)
response_train=np.concatenate(np.concatenate(response,axis=0),axis=0).reshape(-1,300)

In [18]:
final_vector_1=np.concatenate((question_train,response_train),axis=1)
final_vector_1.shape

(1640, 600)

In [19]:
temp1,temp2=[],[]
for i in que_test:
    string=" "
    string=string.join(i)
    temp1.append(string)
for i in resp_test:
    string=" "
    string=string.join(i)
    temp2.append(string)
question,response=pd.Series(temp1),pd.Series(temp2)
question=question.apply(lambda x: get_vector(x))
response=response.apply(lambda x: get_vector(x))
question,response=question.to_numpy(),response.to_numpy()
question,response=question.reshape(-1,1),response.reshape(-1,1)
question_test=np.concatenate(np.concatenate(question,axis=0),axis=0).reshape(-1,300)
response_test=np.concatenate(np.concatenate(response,axis=0),axis=0).reshape(-1,300)

In [20]:
final_vector_1_test=np.concatenate((question_test,response_test),axis=1)
final_vector_1_test.shape

(410, 600)

**PCA(Dimensionality Reduction)**

In [21]:
pca_train=np.concatenate((final_vector,final_vector_1),axis=1)
pca_train.shape

(1640, 1200)

In [22]:
pca_test=np.concatenate((final_vector_test,final_vector_1_test),axis=1)
pca_test.shape

(410, 1200)

In [23]:
scaler = StandardScaler()
pca_train = scaler.fit_transform(pca_train)
pca_test =scaler.fit_transform(pca_test)

In [24]:
pca = PCA(n_components=100)
pca.fit(pca_train)
a=pca.transform(pca_train)

In [25]:
pca = PCA(n_components=100)
pca.fit(pca_test)
b=pca.transform(pca_test)

In [26]:
X_train,X_test,y_train,y_test=a,b,df["type"],df_test["type"]

In [27]:
clf = RandomForestClassifier(n_estimators=10, random_state=30,class_weight="balanced")
clf.fit(X_train,y_train)
predictions=clf.predict(X_test)
print("Accuracy is",accuracy_score(predictions,y_test))
print("F1 Score is",f1_score(predictions,y_test,average='weighted'))
print("Precision Score is",precision_score(predictions,y_test,average="weighted"))
print("Recall Score is",recall_score(predictions,y_test,average="weighted"))


Accuracy is 0.7219512195121951
F1 Score is 0.7838176638176639
Precision Score is 0.8593669768934531
Recall Score is 0.7219512195121951


In [28]:
print(confusion_matrix(y_test, predictions,labels=["answered", "attacked", "irrelevant","agreed"]))

[[295  18   6   1]
 [ 39   0   0   0]
 [ 35   2   1   0]
 [ 13   0   0   0]]


**Cosine Similarity**

In [29]:
import math
import re
from collections import Counter

WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)


cosi=[]
for i in range(len(df)):
  vector1 = text_to_vector(df.question[i])
  vector2 = text_to_vector(df.response[i])
  cosine = get_cosine(vector1, vector2)
  cosi.append(cosine)
print(len(cosi))


1640


In [30]:
cosi=np.array(cosi)
cosi = np.reshape(cosi,(1, cosi.size)).reshape(-1,1)

In [31]:
import math
import re
from collections import Counter

WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)


cosi_test=[]
for i in range(len(df_test)):
  vector1 = text_to_vector(df_test.question[i])
  vector2 = text_to_vector(df_test.response[i])
  cosine = get_cosine(vector1, vector2)
  cosi_test.append(cosine)
print(len(cosi_test))

410


In [32]:
cosi_test=np.array(cosi_test)
cosi_test=np.reshape(cosi_test,(1, cosi_test.size)).reshape(-1,1)

In [33]:
X_train,X_test,y_train,y_test=cosi,cosi_test,df["type"],df_test["type"]

In [34]:
clf=LinearSVC(class_weight="balanced")
clf.fit(X_train,y_train)
predictions=clf.predict(X_test)
print("Accuracy is",accuracy_score(predictions,y_test))
print("F1 Score is",f1_score(predictions,y_test,average='weighted'))
print("Precision Score is",precision_score(predictions,y_test,average="weighted"))
print("Recall Score is",recall_score(predictions,y_test,average="weighted"))


Accuracy is 0.7073170731707317
F1 Score is 0.7377316200526235
Precision Score is 0.7789132702182285
Recall Score is 0.7073170731707317


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
print(confusion_matrix(y_test, predictions,labels=["answered", "attacked", "irrelevant","agreed"]))

[[275   0  45   0]
 [ 35   0   4   0]
 [ 23   0  15   0]
 [  6   0   7   0]]


**Sentiment Analysis**




In [41]:
analyzer = SentimentIntensityAnalyzer()
temp=" "
question = [analyzer.polarity_scores(temp.join(x))['compound'] for x in df_question]
temp=" "
response = [analyzer.polarity_scores(temp.join(x))['compound'] for x in df_response]
question=pd.DataFrame(question)
response=pd.DataFrame(response)


In [42]:
temp=" "
question_test = [analyzer.polarity_scores(temp.join(x))['compound'] for x in df_question_test]
temp=" "
response_test = [analyzer.polarity_scores(temp.join(x))['compound'] for x in df_response_test]
question_test=pd.DataFrame(question_test)
response_test=pd.DataFrame(response_test)


In [43]:
final_vector=np.concatenate((question,response),axis=1)

In [44]:
final_vector_test=np.concatenate((question_test,response_test),axis=1)

**Set 2:Baseline+additional features(classification)**

In [45]:
train=np.concatenate((a,cosi,final_vector),axis=1)
test=np.concatenate((b,cosi_test,final_vector_test),axis=1)

In [46]:
X_train,X_test,y_train,y_test=train,test,df["type"],df_test["type"]

In [47]:
clf=LinearSVC(class_weight="balanced")
clf.fit(X_train,y_train)
predictions=clf.predict(X_test)
print("Accuracy is",accuracy_score(predictions,y_test))
print("F1 Score is",f1_score(predictions,y_test,average='weighted'))
print("Precision Score is",precision_score(predictions,y_test,average="weighted"))
print("Recall Score is",recall_score(predictions,y_test,average="weighted"))


Accuracy is 0.4878048780487805
F1 Score is 0.43841280651110687
Precision Score is 0.4031120395971166
Recall Score is 0.4878048780487805


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [48]:
print(confusion_matrix(y_test, predictions,labels=["answered", "attacked", "irrelevant","agreed"]))

[[194  54  58  14]
 [ 29   3   6   1]
 [ 26   6   3   3]
 [  6   3   4   0]]


In [51]:
clf = RandomForestClassifier(n_estimators=20, random_state=25,class_weight="balanced")
clf.fit(X_train,y_train)
predictions=clf.predict(X_test)
print("Accuracy is",accuracy_score(predictions,y_test))
print("F1 Score is",f1_score(predictions,y_test,average='weighted'))
print("Precision Score is",precision_score(predictions,y_test,average="weighted"))
print("Recall Score is",recall_score(predictions,y_test,average="weighted"))


Accuracy is 0.7536585365853659
F1 Score is 0.8345953668741708
Precision Score is 0.9350076219512194
Recall Score is 0.7536585365853659


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
print(confusion_matrix(y_test, predictions,labels=["answered", "attacked", "irrelevant","agreed"]))

[[309   8   3   0]
 [ 39   0   0   0]
 [ 38   0   0   0]
 [ 11   0   2   0]]


In [53]:
df_final = pd.read_csv('p2_test.csv')
df_final['predicted_type']=predictions
df_final=df_final.rename(columns={"label": "gold_type"})
df_final

,thread_id,question_id,response_id,no_turn_q_id,quoted_q_id,precedent,question,subsequent,response,type,predicted_type
0,232fcl,232fcl,cgsrxd3,NaN,q_27319,NaN,Now if Julie was underage (let's say you and ...,NaN,"\nNo. She's happy, he's happy, who am I to end...",irrelevant,answered
1,221bir,cgikp7a,cginbgy,n_228942,q_28801,"And Egypt...there was a lot of chaos, a lot of...",Did the protests motivate the military to act?,Undoubtedly. But it was still the military tha...,Seems a weird example to use in defence of an ...,attacked,answered
2,20s1x1,cg6ay9r,cg6f18i,n_244708,q_30570,&gt; Hello! Both my major AND my minor are due...,Can you illustrate how a single class in CS w...,Your statement that way more happiness is ach...,"Oh gods yes. First of all, most students aren'...",answered,answered
3,1zyf3k,cfzfpej,cfzg0hy,n_254761,q_31795,"Oh, come on, now you're just trying to misunde...","Didn't I even say, that I am completely and 10...",Let me make it very clear: I am an universal ...,"Sure, but you still obviously have some precon...",irrelevant,answered
4,1yfntu,cfka1pw,cfkar3s,NaN,q_34113,NaN,However if we forget that debate for a minute...,NaN,I'd be willing to guess that this question is ...,attacked,answered
...,...,...,...,...,...,...,...,...,...,...,...
405,2sr2st,cns3qwy,cns4t4s,n_632548,q_78351,"So, just to be clear, if a woman refused to ha...",Am I getting this right?,"But if it's just about the money, would you b...",Yes,answered,answered
406,38x7jw,38x7jw,crykb9x,NaN,q_8321,NaN,Further evidence that merely being overweight ...,NaN,\n,irrelevant,answered
407,2ija3a,cl2zs70,cl33t6t,n_65173,q_8864,Why? Just because it's expected of us?,By who?,This is hardly ever a good reason for doing or...,Maybe I didn't phrase this first point well. B...,answered,irrelevant
408,2hqis0,ckve18f,ckvh0bt,n_70808,q_9521,&gt; You don't have to but you should be able ...,Why are the consequences of everyone taking a...,If everyone became a dental hygienist we'd al...,It's worth considering the societal implicatio...,answered,answered


In [54]:
df_final.to_csv('test_output_proposedsolution.csv')
